### EIA API


In [1]:
# Energy Production and Usage
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### Crude Oil Production

In [2]:
# Crude Oil by State Group/Area
s_list = ['MCRFPCO2', 'MCRFPTX2', 'MCRFPND2', 'MCRFPNM2', 
          'MCRFPUS2', 'MCRFP3FM2']
s_ids = ';'.join([f'PET.{series}.M' for series in s_list])
url = (f'https://api.eia.gov/series/?api_key={eia_key}'+
       f'&series_id={s_ids}&start=198801')
r = requests.get(url).json()
df = pd.DataFrame({i['series_id']: {pd.to_datetime(f'{date}01'): 
                                    val for date, val in i['data']} 
                   for i in r['series']}).sort_index().astype('float')
df.to_csv(data_dir / 'oil_prod_raw.csv', index_label='date')

In [3]:
df = pd.read_csv(data_dir / 'oil_prod_raw.csv', index_col='date', 
                 parse_dates=True)
df['ND_NM_CO'] = df[['PET.MCRFPND2.M', 'PET.MCRFPNM2.M', 'PET.MCRFPCO2.M']].sum(axis=1)
df['USA'] = df['PET.MCRFPUS2.M']
df['TX'] = df['PET.MCRFPTX2.M']
df['GM'] = df['PET.MCRFP3FM2.M']
df['OTH'] = df['USA'] - df['TX'] - df['ND_NM_CO'] - df['GM']

data = df.loc['1989':,['ND_NM_CO', 'TX', 'OTH', 'GM', 'USA']].divide(1000, axis=1)
data.to_csv(data_dir / 'oil_prod.csv', index_label='date')
ltdt = dtxt(data.index[-1])['mon1']
prdt = dtxt(data.index[-13])['mon1']
val = pd.Series({'ltval': data['USA'].iloc[-1], 
           'pryrval': data['USA'].iloc[-13]})
fval = val.apply('{:.1f} million barrels per day'.format)
text = (f'During {ltdt}, the US produced {fval.ltval}, '+
        f'compared to {fval.pryrval} in {prdt}.')
write_txt(text_dir / 'oil_prod.txt', text)
print(text)

During November 2022, the US produced 12.4 million barrels per day, compared to 11.8 million barrels per day in November 2021.


### Electricity Production by Source

In [4]:
# Electricity Production by Major Source
s_list = ['COW', 'NUC', 'TSN', 'HYC', 'HPS', 'NG', 'WND', 'GEO', 'WWW',
         'WAS', 'PEL', 'PC', 'ALL', 'AOR', 'OTH', 'SUN', 'OOG']
s_ids = ';'.join([f'ELEC.GEN.{series}-US-99.M' for series in s_list])
url = (f'https://api.eia.gov/series/?api_key={eia_key}'+
       f'&series_id={s_ids}')
r = requests.get(url).json()
df = pd.DataFrame({i['series_id']: {pd.to_datetime(f'{date}01'): 
                                    val for date, val in i['data']} 
                   for i in r['series']}).sort_index().astype('float')
df.to_csv(data_dir / 'elec_prod_raw.csv', index_label='date')

In [5]:
df = pd.read_csv(data_dir / 'elec_prod_raw.csv', index_col='date', 
                 parse_dates=True)

renew = ['SUN', 'HYC', 'WND', 'GEO', 'WAS', 'WWW']
renewables = [f'ELEC.GEN.{i}-US-99.M' for i in renew]
df['Renewables'] = df[renewables].sum(axis=1)

poth = ['PEL', 'PC', 'OTH', 'OOG', 'HPS']
petoth = [f'ELEC.GEN.{i}-US-99.M' for i in poth]
df['Petroleum and other'] = df[petoth].sum(axis=1)

bio = ['WWW', 'WAS']
biomass = [f'ELEC.GEN.{i}-US-99.M' for i in bio]
df['Biomass'] = df[biomass].sum(axis=1)

rename = {'ELEC.GEN.COW-US-99.M': 'Coal', 
          'ELEC.GEN.NUC-US-99.M': 'Nuclear', 
          'ELEC.GEN.NG-US-99.M': 'Natural Gas',
          'ELEC.GEN.HYC-US-99.M': 'Hydroelectric',
          'ELEC.GEN.SUN-US-99.M': 'Solar',
          'ELEC.GEN.WND-US-99.M': 'Wind',
          'ELEC.GEN.GEO-US-99.M': 'Geothermal'}
df.rename(rename, axis=1, inplace=True)

var = ['Coal', 'Natural Gas', 'Nuclear', 'Renewables', 
       'Petroleum and other', 'Hydroelectric', 'Solar', 
       'Wind', 'Geothermal', 'Biomass']
data = df[var].rolling(12).sum().dropna().divide(1000, axis=1)
data.to_csv(data_dir / 'elec_prod.csv', index_label='date')

popgdp = pd.read_csv(data_dir / 'gdpjobslvl.csv', index_col='date', 
                     parse_dates=True)
popgdp.index = popgdp.index + pd.DateOffset(months=2)
pop = popgdp['pop'].dropna()
popch = (pop.iloc[-1] / pop.loc['2011-03-01'].mean() - 1) * 100
popinc = inc_dec_percent(popch)
gdp = popgdp['gdp'].dropna()
gdpch = (gdp.iloc[-1] / gdp.loc['2011-03-01'].mean() - 1) * 100
gdpinc = inc_dec_percent(gdpch)
major = ['Coal', 'Natural Gas', 'Nuclear', 'Renewables', 
         'Petroleum and other']
elec = data[major].sum(axis=1)
df2 = pd.DataFrame()
df2['elec'] = elec
df2['gdp'] = gdp
df2['pop'] = pop
df2['elecgdp'] = elec / gdp
df2['elecpop'] = elec / pop
df2 = ((df2 / df2.loc['2011-03-01']) - 1) * 100
df2.loc['2011-03-01':].to_csv(data_dir / 'elec.csv', index_label='date')

colors = {'gdp': 'green!60!blue', 
          'pop': 'blue!90!black', 
          'elec': 'red', 
          'elecpop': 'orange', 
          'elecgdp': 'cyan'}

adj = node_adj(df2)
node_file = open(text_dir / 'elec_nodes.txt', 'w')
for series, color in colors.items():
    if series in adj.keys():
        offset = adj[series]
    else:
        offset = 0
    node_file.write(end_node(df2[series].dropna(), color, 
                             percent=True))
node_file.close()
elecgdpch = value_text(df2.elecgdp.dropna().iloc[-1], 'increase_by')
elch = df2.elec.dropna().iloc[-1]
ctxt = ('remained fairly constant at around four trillion '+
        'kilowatt hours')
electxt = value_text(elch) if abs(elch) > 0.5 else ctxt
ltdate = dtxt(df.index[-1])['mon1']
ltval = data[major].iloc[-1].sum()
text = ('Since 2011, annualized total US \\textbf{electricity '+
        f'generation}} has {electxt}. '+
        f'Over the same period, the US population has {popinc} '+
        f'{c_line(colors["pop"])} and real GDP has {gdpinc} '+
        f'{c_line(colors["gdp"])}. As a result, the electricity '+
        f'required to produce a unit of real GDP {elecgdpch} '+
        f'{c_line(colors["elecgdp"])}.') 
write_txt(text_dir / 'elec_prod.txt', text)
print(text)

ngval = data['Natural Gas'].iloc[-1]
coalval = data['Coal'].iloc[-1]
nucval = data['Nuclear'].iloc[-1]
renval = data['Renewables'].iloc[-1]
text = (f'During the 12 months ending {ltdate}, the '+
        f'US generated {ltval:,.0f} billion kilowatt '+
        f'hours of electricity. Of this, {ngval:,.0f} billion '+
        'kilowatt hours were generated using '+
        'natural gas (see\cbox{blue!40!cyan!60!white}), '+
        f'{coalval:,.0f} billion kilowatt hours were generated '+
        f'from coal (see\cbox{{brown}}), {nucval:,.0f} billion '+
        'from nuclear (see\cbox{{red!80!blue!70!white}}), and '+
        f'{renval:,.0f} billion from renewable sources '+
        '(see\cbox{{green!75!blue}}).')
write_txt(text_dir / 'elec_prod2.txt', text)
print(text)

hydval = data['Hydroelectric'].iloc[-1]
bioval = data['Biomass'].iloc[-1]
geoval = data['Geothermal'].iloc[-1]
sunval = data['Solar'].iloc[-1]
wndval = data['Wind'].iloc[-1]
text = (f'Among renewable energy sources, over the year ending '+
        f'{ltdate}, {hydval:,.0f} billion kilowatt hours of '+
        'electricity were generated with conventional hydroelectric '+
        f'(see\cbox{{cyan}}), {bioval:,.0f} billion kilowatt hours '+
        f'were generated from biomass (see\cbox{{brown!92!black}}), '+
        f'{geoval:,.0f} billion were generated from geothermal '+
        f'(see\cbox{{orange}}), {wndval:,.0f} billion from '+
        f'wind (see\cbox{{green!65!blue}}), and {sunval:,.0f} '+
        f'billion from solar (see\cbox{{yellow!70!orange}}).')
write_txt(text_dir / 'elec_prod3.txt', text)
print(text)

Since 2011, annualized total US \textbf{electricity generation} has increased 2.2 percent. Over the same period, the US population has increased by 7.4 percent (see {\color{blue!90!black}\textbf{---}}) and real GDP has increased by 28.1 percent (see {\color{green!60!blue}\textbf{---}}). As a result, the electricity required to produce a unit of real GDP decreased by 19.7 percent (see {\color{cyan}\textbf{---}}).
During the 12 months ending November 2022, the US generated 4,217 billion kilowatt hours of electricity. Of this, 1,677 billion kilowatt hours were generated using natural gas (see\cbox{blue!40!cyan!60!white}), 816 billion kilowatt hours were generated from coal (see\cbox{brown}), 773 billion from nuclear (see\cbox{{red!80!blue!70!white}}), and 914 billion from renewable sources (see\cbox{{green!75!blue}}).
Among renewable energy sources, over the year ending November 2022, 264 billion kilowatt hours of electricity were generated with conventional hydroelectric (see\cbox{cyan})

### Electricity Retail Sales by Sector

In [6]:
# Electricity Retail Sales by Sector
s_list = ['RES', 'COM', 'IND']
s_ids = ';'.join([f'ELEC.SALES.US-{series}.M' for series in s_list])
url = (f'https://api.eia.gov/series/?api_key={eia_key}'+
       f'&series_id={s_ids}&start=200101')
r = requests.get(url).json()
df = pd.DataFrame({i['series_id']: {pd.to_datetime(f'{date}01'): 
                                    val for date, val in i['data']} 
                   for i in r['series']}).sort_index().astype('float')
df.to_csv(data_dir / 'elec_sales_raw.csv', index_label='date')

In [7]:
df = pd.read_csv(data_dir / 'elec_sales_raw.csv', index_col='date', 
                 parse_dates=True) / 1_000

rename = {'ELEC.SALES.US-RES.M': 'Residential', 
          'ELEC.SALES.US-COM.M': 'Commercial',
          'ELEC.SALES.US-IND.M': 'Industrial'}
df.rename(rename, axis=1, inplace=True)

data = df.rolling(12).sum().dropna()
data.to_csv(data_dir / 'elec_sales.csv', index_label='date')

cats = {'Commercial': 'blue!80!cyan', 'Industrial': 
        'black!35!white', 'Residential': 'green!90!black'}
nodes = '\n'.join([end_node(data[cat], col, digits='comma') 
                   for cat, col in cats.items()])
write_txt(text_dir / 'elec_ret_nodes.txt', nodes)

cl = {cat[:3]: c_line(col) for cat, col in cats.items()}
ltdt = dtxt(data.index[-1])['mon1']
dft = data.applymap('{:,.0f} billion'.format)
lt = dft.iloc[-1]
pr = dft.loc['2019-12-01']
text = (f'Over the year ending {ltdt}, retail sales of electricity '+
        f'to the residential sector total {lt.Residential} '+
        f'kilowatt hours, compared to {pr.Residential} '+
        f'during 2019 {cl["Res"]}. Commercial sector '+
        f'electricity sales total {lt.Commercial} kilowatt '+
        f'hours over the year ending {ltdt}, and {pr.Commercial} '+
        f'in 2019 {cl["Com"]}. Industrial '+
        f'sector sales total {lt.Industrial} kilowatt '+
        f'hours in the latest 12 months of data and {pr.Industrial} '+
        f'in 2019 {cl["Ind"]}.')
write_txt(text_dir / 'elec_ret.txt', text)
print(text)

Over the year ending November 2022, retail sales of electricity to the residential sector total 1,506 billion kilowatt hours, compared to 1,440 billion during 2019 (see {\color{green!90!black}\textbf{---}}). Commercial sector electricity sales total 1,368 billion kilowatt hours over the year ending November 2022, and 1,361 billion in 2019 (see {\color{blue!80!cyan}\textbf{---}}). Industrial sector sales total 1,010 billion kilowatt hours in the latest 12 months of data and 1,002 billion in 2019 (see {\color{black!35!white}\textbf{---}}).


### Weekly retail gasoline prices

In [8]:
s_id = 'PET.EMM_EPM0_PTE_NUS_DPG.W'
url = (f'https://api.eia.gov/series/?api_key={eia_key}'+
       f'&series_id={s_id}')
r = requests.get(url).json()
df = (pd.DataFrame({i['series_id']: {pd.to_datetime(date): val 
                                     for date, val in i['data']} 
                    for i in r['series']}).sort_index()
        .astype('float').rename({s_id: 'Value'}, axis=1))
df.to_csv(data_dir / 'gas_price.csv', index_label='date')

node = end_node(df.Value, 'blue', date='d', digits=2, dollar=True, 
                full_year=True)
write_txt(text_dir / 'gas_price_node.txt', node)

# Text
ltdt = dtxt(df.index[-1])['day1']
ltval = df.Value.iloc[-1]
pryrval = df.Value.iloc[-53]
v19 = df.Value.loc['2019'].mean()
v1113 = df.Value.loc['2011': '2013'].mean()
cl = c_line('blue')
ch = value_text(df.Value.diff().iloc[-1], 'increase_of', ptype=None, 
                dollar=True, digits=2)
url = f'https://www.eia.gov/opendata/qb.php?category=241021&sdid={s_id}'
text = (f'On {ltdt}, the US average \href{{{url}}}{{price}} for a gallon of '+
        f'\\textbf{{gasoline}} is \${ltval:.2f} {cl}, {ch} from the week prior. '+
        'This gas price measure, which is the average across '+
        f'formulations, grades, and locations, was \${pryrval:.2f} one '+
        f'year prior, and averaged \${v19:.2f} in 2019. During 2011--2013, '+
        f'the average gas price was \${v1113:.2f}.')
write_txt(text_dir / 'gas_price.txt', text)
print(text)

On January 30, 2023, the US average \href{https://www.eia.gov/opendata/qb.php?category=241021&sdid=PET.EMM_EPM0_PTE_NUS_DPG.W}{price} for a gallon of \textbf{gasoline} is \$3.59 (see {\color{blue}\textbf{---}}), an increase of \$0.07 from the week prior. This gas price measure, which is the average across formulations, grades, and locations, was \$3.46 one year prior, and averaged \$2.69 in 2019. During 2011--2013, the average gas price was \$3.61.


### Oil prices (WTI)

In [9]:
url = (f'https://api.eia.gov/series/?api_key={eia_key}'+
       '&series_id=PET.RCLC1.D&start=19890101')

r = requests.get(url).json()
data = (pd.Series({pd.to_datetime(f'{i[0]}'): i[1] 
                   for i in r['series'][0]['data']})
          .sort_index().to_frame(name='VALUE'))
print('Latest Data:', dtxt(data.index[-1])['day1'])
data.to_csv(data_dir / 'wti_raw.csv', index_label='date')

Latest Data: January 31, 2023


In [10]:
data = pd.read_csv(data_dir / 'wti_raw.csv', index_col='date', 
                   parse_dates=True)
df = data.resample('MS').mean().iloc[:-1]
df.index = df.index + pd.DateOffset(days=14)
lt = data.iloc[-1].to_frame().T
df = pd.concat([df, lt])
df.to_csv(data_dir / 'wti.csv', index_label='date')

oneyr = value_text(df.VALUE.pct_change(12).iloc[-1] * 100)
twoyr = value_text(df.VALUE.pct_change(36).iloc[-1] * 100)

color = 'red!80!purple'
node = end_node(df.VALUE, color, dollar=True, digits=2, date='d', 
                offset=True, full_year=True)
write_txt(text_dir / 'oil_node.txt', node)

maxdt = df.VALUE.idxmax()
maxdtt = dtxt(maxdt)['mon1']
maxval = df.VALUE.max()
ltch = df.loc[maxdt, 'VALUE'] - df.VALUE.iloc[-1]
ldate = dtxt(df.index[-1])['day1']
url = 'https://www.eia.gov/dnav/pet/hist/RCLC1D.htm'
text = (f'On {ldate}, the \href{{{url}}}{{futures price}} '+
        f'for a barrel of west Texas intermediate (WTI) '+
        f'\\textbf{{crude oil}} is \${df.VALUE.iloc[-1]:.2f} {c_line(color)}. '+
        f'Over the past year, this measure of oil prices {oneyr}. Over '+
        f'the past three years, the price {twoyr}. The WTI price is '+
        f'currently \${ltch:.0f} below its peak monthly '+
        f'average price of \${maxval:.0f} per barrel in {maxdtt}.')
write_txt(text_dir / 'wti.txt', text)
print(text)

On January 31, 2023, the \href{https://www.eia.gov/dnav/pet/hist/RCLC1D.htm}{futures price} for a barrel of west Texas intermediate (WTI) \textbf{crude oil} is \$78.87 (see {\color{red!80!purple}\textbf{---}}). Over the past year, this measure of oil prices decreased five percent. Over the past three years, the price increased 36.6 percent. The WTI price is currently \$55 below its peak monthly average price of \$134 per barrel in June 2008.
